<a href="https://colab.research.google.com/github/ciepielajan/folium_map/blob/main/folium_draw_load_save_dual_geopandas_geojson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import folium  # mapa
from folium.plugins import Draw  # rysownia


tiles = "My maps"
zoom_start = 13
location = [50.05,19.95]  # kraków


m = folium.Map(
    location=location, attr=tiles, zoom_start=zoom_start
)


folium.Circle(
    radius=100,
    location=location,
    popup="The Waterfront",
    color="crimson",
    fill=False,
).add_to(m)

draw = Draw(export=True,    # przycisk na mapie
            filename='data.geojson',)     # przycisk na mapie
draw.add_to(m)

m

In [16]:
m.save("index.html")

In [17]:
draw.save("index-draw.html")

`wczytanie pliku i naniesieni na mapę`

In [30]:
# !gdown --id "1sqKY43ZbZIAU87OCzaQF1Ab5x6nFKXfw"  # przykłądowy plik

Downloading...
From: https://drive.google.com/uc?id=1sqKY43ZbZIAU87OCzaQF1Ab5x6nFKXfw
To: /content/data.geojson
100% 1.38k/1.38k [00:00<00:00, 2.47MB/s]


In [48]:
from google.colab import files
uploaded = files.upload()

Saving data (3).geojson to data (3).geojson


In [49]:
import folium

m = folium.Map(location=[50.05,19.95],
               zoom_start=12,
          #  tiles='https://server.arcgisonline.com/arcgis/rest/services/Canvas/World_Dark_Gray_Base/MapServer/tile/{z}/{y}/{x}',
           attr='Własny tytuł mapy')


g = folium.GeoJson(
    "data.geojson",
).add_to(m)

m

`wczytanie pliku do dataframe`

In [30]:
!pip -q install geopandas

     |████████████████████████████████| 1.0MB 17.1MB/s 
     |████████████████████████████████| 14.8MB 262kB/s 
     |████████████████████████████████| 6.5MB 47.1MB/s 


In [31]:
import geopandas

In [50]:
df = geopandas.read_file("data.geojson")
df

,geometry
0,"POLYGON ((19.94524 50.06712, 19.91917 50.05825..."


In [51]:
m = folium.Map([50.05,19.95], zoom_start=13,
               #tiles="cartodbpositron"
               )

folium.GeoJson(df).add_to(m)

m

In [52]:
m.save("index.html")

`DualMap`

In [24]:
import folium.plugins

In [63]:
m = folium.plugins.DualMap(location=(50.05,19.95), tiles=None, zoom_start=12)

folium.TileLayer("openstreetmap").add_to(m.m1)
folium.TileLayer("cartodbpositron").add_to(m.m2)

folium.GeoJson(df).add_to(m.m1)
folium.GeoJson(df).add_to(m.m2)

folium.LayerControl(collapsed=False).add_to(m)

m

In [64]:
m.save('map.html')